## Object tracking using Optical Flow (Lucas-Kanade)

#### Source: Udemy

In [2]:
import numpy as np
import cv2

In [3]:
# Detect 10 best quality corners in the very first frame and track them
corner_track_params = dict(maxCorners=10, qualityLevel=0.3, minDistance=7, blockSize=7)

 ##### Parameters for the Lucas-Kanade Optical Flow function
Trade-off between smaller and larger windows - smaller windows more sensitive to noise and may miss larger movements e.g. the points moving fast. Larger window sizes will catch those movements but not as sensitive.   

Lucas-Kanade method uses an image pyramid (https://en.wikipedia.org/wiki/Pyramid_(image_processing)) and allows us to find optical flow at various resolutions. Level-0 = original image; Level-2 is quarter resolution (MaxLevel=2) 

Criteria: max number of iterations - TERM_CRITERIA_COUNT(10) & Epsilon (EPS) = 0.03. More iterations mean a more exhaustive search for the points. Smaller epsilon means finishing earlier. <b> Speed vs accuracy of tracking <b> 

In [15]:
lk_params = dict(winSize=(200,200), maxLevel=2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

Capture video feed from webcam 

VideoCapture(1) or VideoCapture(0) - front or back camera  

In [72]:
cap = cv2.VideoCapture(1)

ret, prev_frame = cap.read() # capture the first frame of the video to compare to the next frame

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Points to track
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask=None, **corner_track_params)

# mask used to draw lines on the video
mask = np.zeros_like(prev_frame) # creating a numpy array of zeroes as the same shape as the prev_frame

while True:
    
    ret, frame = cap.read() # frame is the current frame - different from prev_frame
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #pass in previous frame, current frame, previous points, None-no current points, params) 
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)
    
    # matching points based on index location. Tracking 10 different points
    good_new = nextPts[status == 1]
    good_prev = prevPts[status == 1]
    
    for i, (new, prev) in enumerate(zip(good_new, good_prev)):
        #calculate x & y positions to draw markers
        
        x_new, y_new = new.ravel() # flattening out an array
        x_prev, y_prev = new.ravel()
        
        #draw green(0,255,0) tracking lines from previous-frame to new-frame
        mask = cv2.line(mask, (x_new, y_new), (x_prev, y_prev), (0,255,0), 3)
        
        #draw a dot on the current frame on the current point we are tracking
        frame = cv2.circle(frame, (x_new, y_new), 8, (0,0,225), -1)
        
        #press escape key to exit
        font = cv2.FONT_HERSHEY_DUPLEX
        position = (10,460)
        message = "Press ESC to Exit"
        menu = cv2.putText(frame, message, position, font, 0.75, (255,255,255))
    
    img = cv2.add(frame, mask, menu)
    cv2.imshow("tracking", img)
    
    #press escape key to exit instructions
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break
    
    # update the previous points to be the current points
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()
    